# TM10007 Assignment template -- ECG data

## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

### Load features and labels into dataframe from csv file 

In [106]:
import zipfile
import os
import pandas as pd

with zipfile.ZipFile('./ecg/ecg_data.zip', 'r') as zip_ref:
    zip_ref.extractall('./ecg/')

data = pd.read_csv('./ecg/ecg_data.csv', index_col=0)

file = './ecg/ecg_data.csv'
if(os.path.exists(file) and os.path.isfile(file)):
  os.remove(file)

print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')

The number of samples: 827
The number of columns: 9001


### Create test-train split

In [107]:
from sklearn.model_selection import train_test_split

y = data.label
X = data.drop('label', axis=1)

# Need to find logical nubmers for test_size and random_state and add this in the report. 
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

### Preprocessing

In [108]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
random_forest = RandomForestClassifier()
pipe = Pipeline(steps=[("scaler", scaler), ("random_forest", random_forest)])

### Gridsearch for optimal hyperparameters Random Forest Classifer 

In [110]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV


parameters = {'n_estimators': [50,100], 'max_depth':[None, 5],}
search = GridSearchCV(pipe, parameters, cv= 3, scoring = 'balanced_accuracy')
search.fit(X_train, y_train)

ValueError: Invalid parameter max_depth for estimator Pipeline(steps=[('scaler', StandardScaler()),
                ('random_forest', RandomForestClassifier())]). Check the list of available parameters with `estimator.get_params().keys()`.

In [99]:
evaluation_grid= pd.DataFrame(clf.cv_results_).sort_values('rank_test_score')
evaluation_grid.drop(['mean_fit_time', 'std_fit_time', 'param_max_depth', 'mean_score_time', 'std_score_time', 'param_n_estimators'], axis=1)

,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
2,"{'max_depth': 5, 'n_estimators': 50}",0.560606,0.544118,0.560606,0.555110,0.007773,1
0,"{'max_depth': None, 'n_estimators': 50}",0.575758,0.519478,0.545455,0.546897,0.022999,2
1,"{'max_depth': None, 'n_estimators': 100}",0.530303,0.537495,0.557295,0.541698,0.011413,3
3,"{'max_depth': 5, 'n_estimators': 100}",0.515152,0.508083,0.545455,0.522896,0.016210,4


### Evaluation of classifier 

In [93]:
from sklearn.metrics import precision_score, confusion_matrix
best_clf = clf.best_estimator_
y_pred = best_clf.predict(X_test)
precision_score(y_test, y_pred)


1.0

In [94]:
pd.DataFrame(confusion_matrix(y_test, y_pred), columns=['normal', 'abnormal'], index= ['normal', 'abnormal'])

,normal,abnormal
normal,227,0
abnormal,43,3
